In [1]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_u1ckzjt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_u1ckzjt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-uigejc6v/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Kaggle". Running its setup...
Updating the package lists...
Installing nvidia-cuda-toolkit, this may take a few minutes...
Source files will be saved in "/tmp/tmpit9o3_v2".


In [2]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <iostream> // for std::ostream
#include <vector>

namespace csc485b{
namespace a2{

using node_t = int;
using edge_t = int2;

using edge_list_t = std::vector< edge_t >;
using node_list_t = std::vector< node_t >;

} // namespace a2
} // namespace csc485b


In [3]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [4]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <cassert>  // for assert()
#include <cstddef>  // std::size_t type
#include <random>   // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a2 {

/**
 * Generates and returns a vector of random edges
 * for a graph `G=(V,E)` with `n=|V|=n` and expected `m=|E|`.
 * Referred to as an Erdős-Rényi graph.
 *
 * @see https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.fast_gnp_random_graph.html#networkx.generators.random_graphs.fast_gnp_random_graph
 */
edge_list_t generate_graph( std::size_t n, std::size_t m )
{
    assert( "At most n(n-1) edges in a simple graph" && m < n * ( n - 1 ) );

    int const probability = ( 100 * m ) / ( n * ( n - 1 ) );

    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20241008;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 100);

    edge_list_t random_edges;
    random_edges.reserve( 2 * m );

    for( node_t u = 0; u < n; ++u )
    {
        for( node_t v = u + 1; v < n; ++v )
        {
            auto const dice_roll = distrib( rng );
            if( dice_roll <= probability )
            {
                random_edges.push_back( make_int2( u, v ) );
                random_edges.push_back( make_int2( v, u ) );
            }
        }
    }

    random_edges.resize( random_edges.size() );


    return random_edges;
}

} // namespace a2
} // namespace csc485b


In [17]:
%%cuda_group_save -g "source" -n "dense_graph.h"
/**
 * The file in which you will implement your DenseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A DenseGraph is optimised for a graph in which the number of edges
 * is close to n(n-1). It is represented using an adjacency matrix.
 */
struct DenseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  node_t * adjacencyMatrix; /** Pointer to an n x n adj. matrix */

  /** Returns number of cells in the adjacency matrix. */
  __device__ __host__ __forceinline__
  std::size_t matrix_size() const { return n * n; }
};

struct DenseGraphCPU {
    std::size_t n;                  // Number of nodes in the graph
    std::vector<int> adjacencyMatrix; // Adjacency matrix stored as a 1D array

    DenseGraphCPU(std::size_t nodes)
        : n(nodes), adjacencyMatrix(nodes * nodes, 0) {}  // Initialize matrix with zeros

    // Helper function to print the adjacency matrix
    void print() const {
        for (std::size_t i = 0; i < n * n; ++i) {
            std::cout << adjacencyMatrix[i] << " ";
        }
        std::cout << std::endl;
    }
};

void build_graph_cpu(DenseGraphCPU& g, const edge_list_t& edge_list) {
    for (const auto& edge : edge_list) {
        int u = edge.x;  // Source node
        int v = edge.y;  // Destination node

        // Update adjacency matrix
        g.adjacencyMatrix[u * g.n + v] = 1;  // Set g[u][v] = 1
        g.adjacencyMatrix[v * g.n + u] = 1;  // Set g[v][u] = 1 (undirected)
    }
}


namespace gpu {


/**
 * Constructs a DenseGraph from an input edge list of m edges.
 *
 * @pre The pointers in DenseGraph g have already been allocated.
 */
__global__
void build_graph(DenseGraph g, int2 const * edge_list, std::size_t m) {
    extern __shared__ int shared_adj_matrix[];  // Shared memory for adjacency matrix

    int tid = threadIdx.x; //local thid
    int idx = blockIdx.x * blockDim.x + tid; //global thid

    // Load a portion of the adjacency matrix into shared memory
    // We will load one row of the adjacency matrix at a time
    if (tid < g.n * g.n) {
        shared_adj_matrix[tid] = g.adjacencyMatrix[tid];
    }
    
    __syncthreads();  // Ensure all threads have loaded the adjacency matrix

    // Ensure the edge processing only happens within bounds
    if (idx < m) {
        // Each thread processes one edge
        int u = edge_list[idx].x;  // Source node
        int v = edge_list[idx].y;  // Destination node

        // Convert the 2D adjacency matrix indices into a 1D array
        std::size_t u_idx = u * g.n + v; // Index for (u, v)
        std::size_t v_idx = v * g.n + u; // Index for (v, u), since the graph is undirected

        // Use atomic operations on shared memory to update the adjacency matrix
        atomicOr(&shared_adj_matrix[u_idx], 1);  // Set g[u][v] = 1
        atomicOr(&shared_adj_matrix[v_idx], 1);  // Set g[v][u] = 1 (undirected edge)
    }
    
    __syncthreads();  // Ensure all atomic operations are done before writing back

    // Write the updated shared memory back to global memory
    if (tid < g.n * g.n) {
        g.adjacencyMatrix[tid] = shared_adj_matrix[tid];
    }
}


/**
  * Repopulates the adjacency matrix as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( DenseGraph g )
{
    // IMPLEMENT ME!
    // square adjacencyMatrix
    // then remove the diagonal and clamp values back to [0,1]
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [15]:
%%cuda_group_save -g "source" -n "sparse_graph.h"
/**
 * The file in which you will implement your SparseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A SparseGraph is optimised for a graph in which the number of edges
 * is close to cn, for a small constanct c. It is represented in CSR format.
 */
struct SparseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  std::size_t m; /**< Number of edges in the graph. */
  node_t * neighbours_start_at; /** Pointer to an n=|V| offset array */
  node_t * neighbours; /** Pointer to an m=|E| array of edge destinations */
};


namespace gpu {

/**
 * Constructs a SparseGraph from an input edge list of m edges.
 *
 * @pre The pointers in SparseGraph g have already been allocated.
 */
__global__
void build_graph( SparseGraph g, edge_t const * edge_list, std::size_t m )
{
    // IMPLEMENT ME!
    return;
}

/**
  * Repopulates the adjacency lists as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( SparseGraph g )
{
    // IMPLEMENT ME!
    // algorithm unknown
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [9]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <chrono>   // for timing
#include <iostream> // std::cout, std::endl
#include <iterator> // std::ostream_iterator
#include <vector>

#include "dense_graph.h"
#include "sparse_graph.h"

#include "data_generator.h"
#include "data_types.h"

/**
 * Runs timing tests on a CUDA graph implementation.
 * Consists of independently constructing the graph and then
 * modifying it to its two-hop neighbourhood.
 */
template < typename DeviceGraph >
void run( DeviceGraph g, csc485b::a2::edge_t const * d_edges, std::size_t m )
{
    cudaDeviceSynchronize();
    auto const build_start = std::chrono::high_resolution_clock::now();

    int threadsPerBlock = 256;
    int blocks = (m + threadsPerBlock - 1) / threadsPerBlock;
    size_t shared_mem_size = g.n * g.n * sizeof(int);  // Size of shared memory
    csc485b::a2::gpu::build_graph<<<blocks, threadsPerBlock, shared_mem_size>>>(g, d_edges, m);

    cudaDeviceSynchronize();
    auto const reachability_start = std::chrono::high_resolution_clock::now();

    // neither does this!
    csc485b::a2::gpu::two_hop_reachability<<< 1, 1 >>>( g );

    cudaDeviceSynchronize();
    auto const end = std::chrono::high_resolution_clock::now();

    std::cout << "Build time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(reachability_start - build_start).count()
              << " us"
              << std::endl;

    std::cout << "Reachability time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(end - reachability_start).count()
              << " us"
              << std::endl;
}

/**
 * Allocates space for a dense graph and then runs the test code on it.
 */
void run_dense( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device DenseGraph
    a2::node_t * d_matrix;
    cudaMalloc( (void**)&d_matrix, sizeof( a2::node_t ) * n * n );
    a2::DenseGraph d_dg{ n, d_matrix };

    run( d_dg, d_edges, m );

    // check output?
    std::vector< a2::node_t > host_matrix( d_dg.matrix_size() );
    a2::DenseGraph dg{ n, host_matrix.data() };
    cudaMemcpy( dg.adjacencyMatrix, d_dg.adjacencyMatrix, sizeof( a2::node_t ) * d_dg.matrix_size(), cudaMemcpyDeviceToHost );
    std::copy( host_matrix.cbegin(), host_matrix.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );

    // clean up
    cudaFree( d_matrix );
}

/**
 * Allocates space for a sparse graph and then runs the test code on it.
 */
void run_sparse( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device SparseGraph
    a2::node_t * d_offsets, * d_neighbours;
    cudaMalloc( (void**)&d_offsets,    sizeof( a2::node_t ) * n );
    cudaMalloc( (void**)&d_neighbours, sizeof( a2::node_t ) * m );
    a2::SparseGraph d_sg{ n, m, d_offsets, d_neighbours };

    run( d_sg, d_edges, m );

    // clean up
    cudaFree( d_neighbours );
    cudaFree( d_offsets );
}

int main()
{
    using namespace csc485b;

    // Create input
    std::size_t constexpr n = 18;
    std::size_t constexpr expected_degree = n >> 1;

    a2::edge_list_t const graph = a2::generate_graph( n, n * expected_degree );
    std::size_t const m = graph.size();

    // lazily echo out input graph
    for( auto const& e : graph )
    {
        std::cout << "(" << e.x << "," << e.y << ") ";
    }
    std::cout << std::endl;

    std::cout << m;
    std::cout << std::endl;

    // allocate and memcpy input to device
    a2::edge_t * d_edges;
    cudaMalloc( (void**)&d_edges, sizeof( a2::edge_t ) * m );
    cudaMemcpyAsync( d_edges, graph.data(), sizeof( a2::edge_t ) * m, cudaMemcpyHostToDevice );

    a2::DenseGraphCPU g_cpu(n);
    build_graph_cpu(g_cpu, graph);
    std::cout << "CPU Adjacency Matrix:" << std::endl;
    g_cpu.print();
    std::cout << std::endl;

    // run your code!
    run_dense ( d_edges, n, m );
    std::cout << std::endl;

    run_sparse( d_edges, n, m );

    return EXIT_SUCCESS;
}

In [18]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

(0,2) (2,0) (0,3) (3,0) (0,4) (4,0) (0,5) (5,0) (0,6) (6,0) (0,9) (9,0) (0,10) (10,0) (0,11) (11,0) (0,13) (13,0) (0,14) (14,0) (0,16) (16,0) (1,4) (4,1) (1,5) (5,1) (1,7) (7,1) (1,8) (8,1) (1,10) (10,1) (1,11) (11,1) (1,12) (12,1) (1,14) (14,1) (1,15) (15,1) (1,16) (16,1) (1,17) (17,1) (2,4) (4,2) (2,5) (5,2) (2,7) (7,2) (2,8) (8,2) (2,9) (9,2) (2,12) (12,2) (2,13) (13,2) (2,15) (15,2) (3,5) (5,3) (3,6) (6,3) (3,9) (9,3) (3,15) (15,3) (4,5) (5,4) (4,8) (8,4) (4,9) (9,4) (4,10) (10,4) (4,11) (11,4) (4,13) (13,4) (4,16) (16,4) (4,17) (17,4) (5,6) (6,5) (5,9) (9,5) (5,11) (11,5) (5,13) (13,5) (5,15) (15,5) (5,16) (16,5) (5,17) (17,5) (6,8) (8,6) (6,9) (9,6) (6,10) (10,6) (6,11) (11,6) (6,12) (12,6) (6,14) (14,6) (7,9) (9,7) (7,10) (10,7) (7,12) (12,7) (7,13) (13,7) (7,15) (15,7) (7,16) (16,7) (7,17) (17,7) (8,9) (9,8) (8,11) (11,8) (8,13) (13,8) (8,14) (14,8) (8,17) (17,8) (9,10) (10,9) (9,11) (11,9) (9,12) (12,9) (9,13) (13,9) (9,16) (16,9) (10,11) (11,10) (10,12) (12,10) (10,14) (14,10